<a href="https://colab.research.google.com/github/alonba/eeg-bss/blob/main/FastICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import mne as mne

ELECTRODES_NUM = 35

root_folder = r"C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss"
train_sub_folder = r"\data\after_ica\train"
test_sub_folder = r"\data\after_ica\test"
file_name = r"\reconstructed_"
file_ext = ".set"

In [8]:
# Fetch the data from the .set file
eeglab_epochs = mne.io.read_epochs_eeglab(root_folder + r'\data\35_electrodes_filtered_epoched.set')

# Create the train set and test set
total_num_of_epochs = len(eeglab_epochs)

train_set_indexes = []
test_set_indexes = []
for i in range(total_num_of_epochs):
    random_num = np.random.rand()
    if (random_num < 0.8):  # 80% chance of being here
        train_set_indexes.append(i)
    else:
        test_set_indexes.append(i)

train_set = eeglab_epochs[train_set_indexes]
test_set = eeglab_epochs[test_set_indexes]

print(len(train_set))
print(len(test_set))

Extracting parameters from C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss\data\35_electrodes_filtered_epoched.set...
Not setting metadata
460 matching events found
No baseline correction applied
0 projection items activated
Ready.
368
92


In [9]:
# Export the Data
mne.export.export_epochs(root_folder + r'\data\train_set.set', train_set, overwrite=True)
mne.export.export_epochs(root_folder + r'\data\test_set.set', test_set, overwrite=True)

In [10]:
train_set = mne.io.read_epochs_eeglab(root_folder + r'\data\train_set.set')
train_ica = mne.preprocessing.ICA(n_components = ELECTRODES_NUM)  # Defaults to FastICA
train_ica.fit(train_set)
TRAIN_SET_NUM_OF_TRIALS, _ , NUM_OF_TIME_SAMPLES_IN_TRIAL = train_set.get_data().shape

test_set = mne.io.read_epochs_eeglab(root_folder + r'\data\test_set.set')
test_ica = mne.preprocessing.ICA(n_components = ELECTRODES_NUM)  # Defaults to FastICA
test_ica.fit(test_set)
TEST_SET_NUM_OF_TRIALS = test_set.get_data().shape[0]

Extracting parameters from C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss\data\train_set.set...
Not setting metadata
368 matching events found
No baseline correction applied
0 projection items activated
Ready.
Fitting ICA to data using 35 channels (please be patient, this may take a while)


C:\Users\alonb\AppData\Local\Temp\ipykernel_524\2097318266.py:3: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  train_ica.fit(train_set)


Selecting by number: 35 components
Fitting ICA took 153.2s.
Extracting parameters from C:\Users\alonb\OneDrive - Technion\טכניון\סמסטר ט' - אביב 2022\פרוייקט א\eeg-bss\data\test_set.set...
Not setting metadata
92 matching events found
No baseline correction applied


c:\Users\alonb\anaconda3\envs\eeg_bss\lib\site-packages\sklearn\decomposition\_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


0 projection items activated
Ready.
Fitting ICA to data using 35 channels (please be patient, this may take a while)
Selecting by number: 35 components


C:\Users\alonb\AppData\Local\Temp\ipykernel_524\2097318266.py:8: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  test_ica.fit(test_set)


Fitting ICA took 28.4s.


c:\Users\alonb\anaconda3\envs\eeg_bss\lib\site-packages\sklearn\decomposition\_fastica.py:116: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


In [13]:
train_data = train_set.get_data()
train_data_swapped = np.swapaxes(train_data,0,1)
train_data_reshaped = train_data_swapped.reshape(ELECTRODES_NUM,TRAIN_SET_NUM_OF_TRIALS*NUM_OF_TIME_SAMPLES_IN_TRIAL)
cov = np.cov(train_data_reshaped)
x=2

In [67]:
train_set_left = train_set['Left']
train_set_right = train_set['Right']
train_set_no_move = train_set['NoMove']

TRAIN_LEFT_NUM_OF_TRIALS = len(train_set_left.selection)
TRAIN_RIGHT_NUM_OF_TRIALS = len(train_set_right.selection)
TRAIN_NO_MOVE_NUM_OF_TRIALS = len(train_set_no_move.selection)

train_left_data = train_set_left.get_data()
train_right_data = train_set_right.get_data()
train_no_move_data = train_set_no_move.get_data()

train_left_data_swapped = np.swapaxes(train_left_data,0,1)
train_right_data_swapped = np.swapaxes(train_right_data,0,1)
train_no_move_data_swapped = np.swapaxes(train_no_move_data,0,1)

reshaped_train_set_left = train_left_data_swapped.reshape((ELECTRODES_NUM, TRAIN_LEFT_NUM_OF_TRIALS * NUM_OF_TIME_SAMPLES_IN_TRIAL))
reshaped_train_set_right = train_right_data_swapped.reshape((ELECTRODES_NUM, TRAIN_RIGHT_NUM_OF_TRIALS * NUM_OF_TIME_SAMPLES_IN_TRIAL))
reshaped_train_set_no_move = train_no_move_data_swapped.reshape((ELECTRODES_NUM, TRAIN_NO_MOVE_NUM_OF_TRIALS * NUM_OF_TIME_SAMPLES_IN_TRIAL))

train_ICs_left = train_ica.unmixing_matrix_ @ reshaped_train_set_left
train_ICs_right = train_ica.unmixing_matrix_ @ reshaped_train_set_right
train_ICs_no_move = train_ica.unmixing_matrix_ @ reshaped_train_set_no_move

train_ICs_left_reshaped = train_ICs_left.reshape((ELECTRODES_NUM, TRAIN_LEFT_NUM_OF_TRIALS, NUM_OF_TIME_SAMPLES_IN_TRIAL))
train_ICs_right_reshaped = train_ICs_right.reshape((ELECTRODES_NUM, TRAIN_RIGHT_NUM_OF_TRIALS, NUM_OF_TIME_SAMPLES_IN_TRIAL))
train_ICs_no_move_reshaped = train_ICs_no_move.reshape((ELECTRODES_NUM, TRAIN_NO_MOVE_NUM_OF_TRIALS, NUM_OF_TIME_SAMPLES_IN_TRIAL))

train_ICs = np.concatenate((train_ICs_left_reshaped, train_ICs_right_reshaped, train_ICs_no_move_reshaped), axis=1)

In [68]:
test_set_left = test_set['Left']
test_set_right = test_set['Right']
test_set_no_move = test_set['NoMove']

TEST_LEFT_NUM_OF_TRIALS = len(test_set_left.selection)
TEST_RIGHT_NUM_OF_TRIALS = len(test_set_right.selection)
TEST_NO_MOVE_NUM_OF_TRIALS = len(test_set_no_move.selection)

test_left_data = test_set_left.get_data()
test_right_data = test_set_right.get_data()
test_no_move_data = test_set_no_move.get_data()

test_left_data_swapped = np.swapaxes(test_left_data,0,1)
test_right_data_swapped = np.swapaxes(test_right_data,0,1)
test_no_move_data_swapped = np.swapaxes(test_no_move_data,0,1)

reshaped_test_set_left = test_left_data_swapped.reshape((ELECTRODES_NUM, TEST_LEFT_NUM_OF_TRIALS * NUM_OF_TIME_SAMPLES_IN_TRIAL))
reshaped_test_set_right = test_right_data_swapped.reshape((ELECTRODES_NUM, TEST_RIGHT_NUM_OF_TRIALS * NUM_OF_TIME_SAMPLES_IN_TRIAL))
reshaped_test_set_no_move = test_no_move_data_swapped.reshape((ELECTRODES_NUM, TEST_NO_MOVE_NUM_OF_TRIALS * NUM_OF_TIME_SAMPLES_IN_TRIAL))

test_ICs_left = test_ica.unmixing_matrix_ @ reshaped_test_set_left
test_ICs_right = test_ica.unmixing_matrix_ @ reshaped_test_set_right
test_ICs_no_move = test_ica.unmixing_matrix_ @ reshaped_test_set_no_move

test_ICs_left_reshaped = test_ICs_left.reshape((ELECTRODES_NUM, TEST_LEFT_NUM_OF_TRIALS, NUM_OF_TIME_SAMPLES_IN_TRIAL))
test_ICs_right_reshaped = test_ICs_right.reshape((ELECTRODES_NUM, TEST_RIGHT_NUM_OF_TRIALS, NUM_OF_TIME_SAMPLES_IN_TRIAL))
test_ICs_no_move_reshaped = test_ICs_no_move.reshape((ELECTRODES_NUM, TEST_NO_MOVE_NUM_OF_TRIALS, NUM_OF_TIME_SAMPLES_IN_TRIAL))

test_ICs = np.concatenate((test_ICs_left_reshaped, test_ICs_right_reshaped, test_ICs_no_move_reshaped), axis=1)

In [72]:
train_labels_left = np.full((TRAIN_LEFT_NUM_OF_TRIALS),1)
train_labels_right = np.full((TRAIN_RIGHT_NUM_OF_TRIALS),2)
train_labels_no_move = np.full((TRAIN_NO_MOVE_NUM_OF_TRIALS),3)
train_labels = np.concatenate((train_labels_left, train_labels_right, train_labels_no_move))

test_labels_left = np.full((TEST_LEFT_NUM_OF_TRIALS),1)
test_labels_right = np.full((TEST_RIGHT_NUM_OF_TRIALS),2)
test_labels_no_move = np.full((TEST_NO_MOVE_NUM_OF_TRIALS),3)
test_labels = np.concatenate((test_labels_left, test_labels_right, test_labels_no_move))

In [73]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_ICs[0], train_labels)
clf.predict([test_ICs[0,0]])


array([2])

In [74]:
print(test_labels[0])

1


In [ ]:
# train_left_reconstructed = []
# train_right_reconstructed = []
# train_no_move_reconstructed = []
# for i in range(ELECTRODES_NUM):
#     tmp_train_left = train_set_left.copy()
#     tmp_train_right = train_set_right.copy()
#     tmp_train_no_move = train_set_no_move.copy()
#     train_left_reconstructed.append(train_ica.apply(tmp_train_left,include=i))
#     train_right_reconstructed.append(train_ica.apply(tmp_train_right,include=i))
#     train_no_move_reconstructed.append(train_ica.apply(tmp_train_no_move,include=i))

In [21]:
### Export the projected data to .set files, EEGLAB format

# for i in range(ELECTRODES_NUM):
#     mne.export.export_epochs(root_folder+train_sub_folder+r"\left"+file_name+str(i)+file_ext,train_left_reconstructed[i])
#     mne.export.export_epochs(root_folder+train_sub_folder+r"\right"+file_name+str(i)+file_ext,train_right_reconstructed[i])
#     mne.export.export_epochs(root_folder+train_sub_folder+r"\no_move"+file_name+str(i)+file_ext,train_no_move_reconstructed[i])

In [ ]:
# test_set = mne.io.read_epochs_eeglab(root_folder+r'\data\test_set.set')
# test_ica = mne.preprocessing.ICA(n_components=ELECTRODES_NUM)  # Defaults to FastICA
# test_ica.fit(test_set)

# test_set_left=test_set['Left']
# test_set_right=test_set['Right']
# test_set_no_move=test_set['NoMove']

# test_left_reconstructed = []
# test_right_reconstructed = []
# test_no_move_reconstructed = []
# for i in range(ELECTRODES_NUM):
#     tmp_test_left = test_set_left.copy()
#     tmp_test_right = test_set_right.copy()
#     tmp_test_no_move = test_set_no_move.copy()
#     test_left_reconstructed.append(test_ica.apply(tmp_test_left,include=i))
#     test_right_reconstructed.append(test_ica.apply(tmp_test_right,include=i))
#     test_no_move_reconstructed.append(test_ica.apply(tmp_test_no_move,include=i))
    
# # Export the projected data to .set files, EEGLAB format
# for i in range(ELECTRODES_NUM):
#     mne.export.export_epochs(root_folder+test_sub_folder+r"\left"+file_name+str(i)+file_ext,test_left_reconstructed[i])
#     mne.export.export_epochs(root_folder+test_sub_folder+r"\right"+file_name+str(i)+file_ext,test_right_reconstructed[i])
#     mne.export.export_epochs(root_folder+test_sub_folder+r"\no_move"+file_name+str(i)+file_ext,test_no_move_reconstructed[i])

In [ ]:
# ### Import the projected data

# train_left_reconstructed=[]
# train_right_reconstructed=[]
# train_no_move_reconstructed=[]
# test_left_reconstructed=[]
# test_right_reconstructed=[]
# test_no_move_reconstructed=[]
# for i in range(ELECTRODES_NUM):
#     train_left_reconstructed.append(mne.read_epochs_eeglab(root_folder+train_sub_folder+r"\left"+file_name+str(i)+file_ext))
#     train_right_reconstructed.append(mne.read_epochs_eeglab(root_folder+train_sub_folder+r"\right"+file_name+str(i)+file_ext))
#     train_no_move_reconstructed.append(mne.read_epochs_eeglab(root_folder+train_sub_folder+r"\no_move"+file_name+str(i)+file_ext))
#     test_left_reconstructed.append(mne.read_epochs_eeglab(root_folder+test_sub_folder+r"\left"+file_name+str(i)+file_ext))
#     test_right_reconstructed.append(mne.read_epochs_eeglab(root_folder+test_sub_folder+r"\right"+file_name+str(i)+file_ext))
#     test_no_move_reconstructed.append(mne.read_epochs_eeglab(root_folder+test_sub_folder+r"\no_move"+file_name+str(i)+file_ext))